In [ ]:
# genera

Some scripts to run data collected in and S3 bucket in csv format in pyspark.

## PySpark 
For PySpark, enter your Python code in the cell below. You can run the cell by hitting "Run" from the above toolbar. Example Python code: <br>


In [ ]:
print("hello world")

## Spark SQL
Enter %%sql in the first line and your SQL query in subsquent lines. You can run the query by hitting "Run" from the above toolbar. Example SQL query (you will need to replace &lt;database&gt; and &lt;table&gt; with your specific database and table): <br>


In [ ]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('s3a://covid19-dataanalytics-mulder/data/time_series_covid19_confirmed_global.csv')


In [ ]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int' or t[1] == 'double']
df.select(numeric_features).toPandas().corr()


In [ ]:
pandas_df = df.toPandas()


In [ ]:
pandas_df.describe()

In [ ]:
china_df = pandas_df[pandas_df["Country/Region"] == "China"]

In [ ]:
china_df.describe()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df_melted = china_df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"],
                    var_name="Date",
                    value_name="Cases")
df_melted['Date'] = pd.to_datetime(df_melted['Date'])

# Total Cases Over Time
total_cases_over_time = df_melted.groupby('Date').sum()['Cases']
plt.figure(figsize=(12,6))
total_cases_over_time.plot()
plt.title('Total COVID-19 Cases Over Time in China')
plt.xlabel('Date')
plt.ylabel('Total Cases')
plt.show()

sample_provinces = df_melted[df_melted['Province/State'].isin(['Anhui', 'Beijing', 'Hubei'])]
province_cases_over_time = sample_provinces.pivot_table(index='Date', columns='Province/State', values='Cases')
province_cases_over_time.plot(figsize=(12,6))
plt.title('COVID-19 Cases Over Time in Selected Provinces of China')
plt.xlabel('Date')
plt.ylabel('Number of Cases')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_melted['Cases'] = pd.to_numeric(df_melted['Cases'], errors='coerce')
df_melted['Date'] = pd.to_datetime(df_melted['Date'], errors='coerce')

df_melted.dropna(subset=['Cases', 'Date'], inplace=True)

total_cases_over_time = df_melted.groupby('Date').sum()['Cases']
plt.figure(figsize=(12,6))
total_cases_over_time.plot()
plt.title('Total COVID-19 Cases Over Time in China')
plt.xlabel('Date')
plt.ylabel('Total Cases')
plt.show()
u

sample_provinces = df_melted[df_melted['Province/State'].isin(['Anhui', 'Beijing', 'Hubei'])]
province_cases_over_time = sample_provinces.pivot_table(index='Date', columns='Province/State', values='Cases')
province_cases_over_time.plot(figsize=(12,6))
plt.title('COVID-19 Cases Over Time in Selected Provinces of China')
plt.xlabel('Date')
plt.ylabel('Number of Cases')
plt.show()
%matplot plt

![Local Image](./assets/china-covid.png)

In [ ]:
mexico_df = pandas_df[pandas_df["Country/Region"] == "Mexico"]
df_melted_mexico = mexico_df.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"],
                    var_name="Date",
                    value_name="Cases")


In [ ]:
import matplotlib.pyplot as plt

df_melted_mexico['derivative'] = df_melted_mexico['Cases'].diff()

plt.figure(figsize=(15, 7))

plt.plot(df_melted_mexico['Date'], df_melted_mexico['Cases'], label='Total Cases', color='blue')

plt.plot(df_melted_mexico['Date'], df_melted_mexico['derivative']*50, label='Daily Changes in Cases', color='red')

plt.xlabel('Date')
plt.ylabel('Number of Cases')
plt.title('COVID-19 Total Cases and Daily Changes in Mexico')

plt.legend()

plt.show()

%matplot plt

![Local Image](./assets/mexico_covid.png)